# Checking and Installing Libraries

Make sure that AnVIL and DiffBind packages are installed

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager", repos = "https://cran.r-project.org")
if (!requireNamespace("AnVIL", quietly = TRUE))
    BiocManager::install("AnVIL")
if (!requireNamespace("DiffBind", quietly = TRUE))
    AnVIL::install("DiffBind")

In [ ]:
suppressMessages(library(AnVIL))
suppressMessages(library(DiffBind))

# Load Workspace Tables

Load in the particular table that you want and read in the DBA object

In [ ]:
avtables()
table <- avtable("MM")
table

In [ ]:
get.file <- function(path){
    dest <- getwd()
    gsutil_cp(path, dest)
    name <- basename(path)
    return(name)
}

file_path <- get.file(table$Robj[table$MM_id == 'Donor_1062_all_vs_MM_H3K27ac'])
data <- readRDS(file_path)
data

# Run Differential Analysis

Specify the contrast and the label you are interested in. Then run normalization, contrast, and analyze.

In [ ]:
contrast <- DBA_FACTOR
label <- DBA_CONDITION

In [ ]:
library(parallel)
data$config$cores <- detectCores()

In [ ]:
norm <- dba.normalize(data)

In [ ]:
cont <- dba.contrast(norm, categories=contrast, minMembers=2)

In [ ]:
diffs <- dba.analyze(cont, method=DBA_DESEQ2, bParallel=F)
# diffs <- dba.analyze(cont, method=DBA_EDGER)

# Plots

Different plots that can be generated after differential analysis has been run.

In [ ]:
plot(diffs, main='', sub='')

In [ ]:
dba.plotPCA(diffs, contrast=contrast, label=label)

In [ ]:
peaks <- dba.report(diffs)
peaks_df <- as(peaks,"data.frame")
hist(peaks_df$Fold, breaks=100)

In [ ]:
dba.plotMA(diffs)

In [ ]:
dba.plotVolcano(diffs)

In [ ]:
dba.plotHeatmap(diffs, contrast=contrast, correlations=FALSE)